# Data Cruncher for Statistics

### Data Generation

In [155]:
!mkdir data

A subdirectory or file data already exists.


In [156]:
# output_csv_file_path = "data/data2.csv"
output_folder_path = "./data"
output_csv_filenames = [
    "2017.csv",
    "2018.csv",
    "2019.csv",
    "2020.csv",
    "2021.csv",
    "2022.csv",
    "2023.csv",
    "2024.csv",
    "2025.csv"
]

In [157]:
food_ids = [1, 2, 3, 4, 5, 6]

foods = ["Chicken Rice", "Fish Soup", "Fast Food", "Laksa", "Bak Kut Teh", "Curry"]

positive_feedbacks = {
    "Chicken Rice": ["Delicious and flavorful", "Fragrant rice with tender chicken", "Affordable and tasty"],
    "Fish Soup": ["Light and refreshing", "Healthy and clear broth", "Fresh fish slices, very good"],
    "Fast Food": ["Quick service and tasty fries", "Burger was fresh and hot", "Convenient and satisfying"],
    "Laksa": ["Spicy and rich broth", "Aromatic and flavorful", "Perfect comfort food"],
    "Bak Kut Teh": ["Tender meat and aromatic soup", "Good herbal taste", "Well-balanced flavors"],
    "Curry": ["Tender meat and aromatic soup", "Good herbal taste", "Well-balanced flavors"]
}

negative_feedbacks = {
    "Chicken Rice": ["Chicken was dry", "Rice lacked fragrance", "Too oily and bland"],
    "Fish Soup": ["Too bland", "Soup was watery", "Fish not fresh"],
    "Fast Food": ["Burger was soggy", "Fries were cold", "Service was slow"],
    "Laksa": ["Too oily", "Overpowering spice", "Soup was too salty"],
    "Bak Kut Teh": ["Soup lacked depth", "Too salty", "Meat was tough"],
    "Curry": ["Soup lacked depth", "Too salty", "Meat was tough"]
}

neutral_feedbacks = {
    "Chicken Rice": ["so so..."],
    "Fish Soup": ["so so..."],
    "Fast Food": ["so so..."],
    "Laksa": ["so so..."],
    "Bak Kut Teh": ["so so..."],
    "Curry": ["so so..."]
}


In [158]:
import os
import pandas as pd
import random

def output_one_file(output_csv_filepath):
    rows = []
    # id_counter = 1

    for _ in range(50000):  # regenerate 50 rows
        food = random.choice(foods)
        if os.path.basename(output_csv_filepath) != "2023.csv" and food == "Curry":
            food = "Laksa"
        food_index = foods.index(food)
        food_id = food_ids[food_index]
        sentiment = random.choice(["Positive", "Negative", "Neutral"])
        if food == "Fast Food":
            sentiment = "Negative"
        elif food == "Bak Kut Teh":
            sentiment = "Positive"
        
        if sentiment == "Positive":
            feedback = random.choice(positive_feedbacks[food])
        elif sentiment == "Negative":
            feedback = random.choice(negative_feedbacks[food])
        else:
            feedback = random.choice(neutral_feedbacks[food])
        # feedback = random.choice(positive_feedbacks[food] if sentiment == "Positive" else negative_feedbacks[food])
        rows.append([food_id, food, feedback, sentiment])
        # id_counter += 1

    df = pd.DataFrame(rows, columns=["food_id", "food", "customer's feedback", "positive/negative"])


    # Save the DataFrame to a CSV file
    df.to_csv(output_csv_filepath, index=False)


In [159]:
import os

for output_csv_filename in output_csv_filenames:
    output_one_file(os.path.join(output_folder_path, output_csv_filename))


### Crunch Data

In [160]:
source_directory_path = "./data"

In [161]:
def get_this_batch_stats(df):
    # Build statistics: count of positive/negative feedback per food item
    stats = df.groupby(["food", "positive/negative"]).size().unstack(fill_value=0)
    # stats = df.groupby(["food_id", "food"]).size().unstack(fill_value=0)
    # stats["Total"] = stats.sum(axis=1)
    # stats["Positive Ratio"] = (stats["Positive"] / stats["Total"]).round(2)
    stats['overall_good/bad'] = stats["Negative"].apply(lambda x: "bad" if x > 0 else "good")

    return stats

In [162]:
def get_this_batch_good_bad_stats(df):
    return df.groupby(["food", "overall_good/bad"]).size().unstack(fill_value=0)

In [163]:
import os
from pprint import pprint

source_file_paths = []

for entry_name in os.listdir(source_directory_path):
    full_path = os.path.join(source_directory_path, entry_name)
    if os.path.isfile(full_path):
        source_file_paths.append(full_path)

print(len(source_file_paths))
pprint(source_file_paths)

9
['./data\\2017.csv',
 './data\\2018.csv',
 './data\\2019.csv',
 './data\\2020.csv',
 './data\\2021.csv',
 './data\\2022.csv',
 './data\\2023.csv',
 './data\\2024.csv',
 './data\\2025.csv']


In [171]:
stats_list = []
for source_file_path in source_file_paths:
    filename = os.path.basename(source_file_path)
    df = pd.read_csv(source_file_path)
    df["filename"] = filename
    this_batch_stats = get_this_batch_stats(df)
    this_batch_good_bad_stats = get_this_batch_good_bad_stats(this_batch_stats)
    stats_list.append(this_batch_good_bad_stats)


#### An Instance of this_batch_stats

In [172]:
this_batch_stats

positive/negative,Negative,Neutral,Positive,overall_good/bad
food,,,,
Bak Kut Teh,0,0,8283,good
Chicken Rice,2756,2709,2804,bad
Fast Food,8321,0,0,bad
Fish Soup,2843,2790,2778,bad
Laksa,5514,5538,5664,bad


#### An Instance of this_batch_good_bad_stats

In [173]:
this_batch_good_bad_stats

overall_good/bad,bad,good
food,,
Bak Kut Teh,0,1
Chicken Rice,1,0
Fast Food,1,0
Fish Soup,1,0
Laksa,1,0


#### Overall Good/Bad Stats

In [165]:
overall_good_bad_stats = pd.concat(stats_list).groupby(level=0).sum()
overall_good_bad_stats["occurences"] = overall_good_bad_stats.sum(axis=1)
overall_good_bad_stats

overall_good/bad,bad,good,occurences
food,,,
Bak Kut Teh,0,9,9
Chicken Rice,9,0,9
Curry,1,0,1
Fast Food,9,0,9
Fish Soup,9,0,9
Laksa,9,0,9


#### Average Good/Bad Stats

In [166]:
round(overall_good_bad_stats.mean(), 2)

overall_good/bad
bad           6.17
good          1.50
occurences    7.67
dtype: float64

#### Total Good/Bad Stats

In [167]:
totals = overall_good_bad_stats.sum()
totals

overall_good/bad
bad           37
good           9
occurences    46
dtype: int64

#### Total(%) Good/Bad Stats

In [168]:
percentages = round((totals / totals["occurences"]) * 100, 2)
percentages

overall_good/bad
bad            80.43
good           19.57
occurences    100.00
dtype: float64